G7_TIME-SERIES ANALYSIS OF RETAIL SALES DATA 

### Preparing the Data

In [2]:
#Loading datasets and some necessary libraries


import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.graphics.tsaplots import acf, pacf, plot_acf, plot_pacf
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.stattools import adfuller
from IPython.display import display
from sklearn.metrics import mean_squared_error







In [3]:
sales_df = pd.read_csv('data/sales.csv')
ext_variables_df = pd.read_csv('data/External Variables.csv')

# Converting date column to datetime format
ext_variables_df['Date'] = pd.to_datetime(ext_variables_df['Date'])
sales_df['Date'] = pd.to_datetime(sales_df['Date'])

In [ ]:
#Checking the size of our dataframes
print(ext_variables_df.shape)
print(sales_df.shape)

In [ ]:
#Merging the dataframe

df = pd.merge(sales_df,ext_variables_df, on=['Store','Date', 'IsHoliday'], how='left')

# describing the final dataframe that we have
print(df.dtypes)
df.head()

* we have some NaN values in some numeric columns such as MarkDown1 etc, so we will fill them with zeros

In [ ]:
df.fillna(0, inplace= True)
df.head()

### Exploratory Data Analysis

#### Utilities function for EDA  to avoid code redundancy.

In [8]:
# This function takes time series as an input and prints the result of Dickey-Fuller test as a result 

'''  initial idea is to use an AR model with linear regression since we have external factors to consider and 
since The Dickey Fuller test checks the null hypothesis if a unit root is present in an autoregressive mode we choose to 
test the stationarity using this test'''

def adf_test(weekly_sales):
    #Perform Dickey-Fuller test:
    #print ('Results of Dickey-Fuller Test:')
    dftest = adfuller(weekly_sales, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
           dfoutput['Critical Value (%s)'%key] = value
    
    #print (dfoutput)
    return dfoutput

In [9]:
# This function will take store_id and data_frame as inputs and returns all weekly sales for specified store as time series
def get_Weekly_sales_as_time_series_by_store_id(store_id, data_frame):
    df_result=data_frame.where(data_frame['Store'] == store_id)
    df_result=df_result.dropna()
    df_result=df_result.groupby(by=['Date'], as_index=False)['Weekly_Sales'].sum()
    df_result = df_result.set_index('Date')
    df_result.sort_values('Date', inplace=True)
    return df_result

In [10]:
# This function will take store_id and data_frame as inputs and returns all external variables for specified store
def get_external_variables_by_store_id(store_id, data_frame):
    external_variables_df=data_frame.where(data_frame['Store'] == store_id)
    external_variables_df=external_variables_df.dropna()
    external_variables_df=external_variables_df.groupby(by=['Date'], as_index=False)[['Temperature', 'Fuel_Price', 'CPI', \
                                                        'Unemployment', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4',\
                                                                                      'MarkDown5']].mean()
    external_variables_df = external_variables_df.set_index('Date')
    external_variables_df.sort_values('Date', inplace=True)
    return external_variables_df

#### Plot Functions to Visualize

In [11]:
### This function plots sales of a fiven store against time, helps us to visualize the sales and to some extent if the
### series is stationary

def plot_sales(time_series):
    
    _ = plt.figure(figsize=(20,5))
    _ = plt.plot(time_series.index, time_series.values, label = 'Time Series')
    _ = rolling_mean = time_series.rolling(8).mean()
    _ = rolling_mean = rolling_mean.dropna()
    _ = plt.plot(rolling_mean, color='red', label='Rolling Mean')
    _ = plt.ylabel("Sales ($)", fontsize = 15)
    _ = plt.xlabel("Date", fontsize = 15)
    _ = plt.legend(fontsize = 15)
    plt.show()
    return None

In [12]:
### This function plots the pacf and acf curves for the store which will 
### help us in identifying AR parameters for our time series model

def plot_acf_and_pacf(time_series):
    
    fig, axes = plt.subplots(1,2, figsize=(20,5))
    plot_acf(time_series.values,lags=70, ax=axes[0])
    plot_pacf(time_series.values,lags=70, ax=axes[1])
    plt.show()

#### Effect of external variables on Sales

We will check the effect of external variables on weekly sales of a store, store 42 for example. We will use the functions we wrote to plot the sales curve. 

#### Plots of External Variables with time

In [ ]:
ext_store_42 = get_external_variables_by_store_id(42, df) 
store_42_ts = get_Weekly_sales_as_time_series_by_store_id(42, df) 
ext_store_42['Date'] = ext_store_42.index
ext_store_42['Weekly_Sales'] = store_42_ts['Weekly_Sales']
ext_store_42[['Date', 'Unemployment', 'Fuel_Price', 'Temperature', 'CPI', 'MarkDown1', \
    'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'Weekly_Sales']].plot(x='Date', subplots=True, figsize=(20,30))

plt.show()

#### Stationarity Test for Stores

As part of our EDA we check the stationarity of stores through ADF function we have written earlier. 

In [15]:
all_stores = list(set(df['Store']))
stores_dicky_fuller = []
stationary_stores = []

for store in all_stores:
    store_sales = get_Weekly_sales_as_time_series_by_store_id(store, df)
    adf_test_result = adf_test(store_sales['Weekly_Sales'])
    (store, pvalue, test_statistic) = store,adf_test_result['p-value'],adf_test_result['Test Statistic']
    stores_dicky_fuller.append((store, pvalue, test_statistic))
    if (pvalue < 0.05) and (adf_test_result['Test Statistic'] < adf_test_result['Critical Value (1%)']):
        stationary_stores.append(store)
        


In [ ]:
print('Stationary Stores:', stationary_stores)
non_stationary_stores = [store_num for store_num in all_stores if store_num not in stationary_stores]
print('')
print('Non Stationary Stores:', non_stationary_stores)

Now that we have a list for stationary and non-stationary stores, we visualize the sales figures for some of them. 

We start off by picking store '42' for stationary and store '30' for non-stationary

In [ ]:
store_42_ts = get_Weekly_sales_as_time_series_by_store_id(42, df)
store_42_external_variables = get_external_variables_by_store_id(42, df)
plot_sales(store_42_ts)

In [ ]:
store_30_ts = get_Weekly_sales_as_time_series_by_store_id(30, df)
store_30_external_variables = get_external_variables_by_store_id(30, df)
plot_sales(store_30_ts)

From the above visualization we see that both have sort of a varying rolling mean but the fluctuation is more apparent in store_30 and hence adf test validates non-stationarity of the store. As we see from these time series plots it is very diffcult to just go by a mere visual appeal to test stationarity and hence the adf test helps. 



Displaying the ADF test results for both the stores. 


In [ ]:
c = adf_test(store_42_ts["Weekly_Sales"])
print(c)
c['p-value']

In [ ]:
c = adf_test(store_30_ts["Weekly_Sales"])
print(c)
c['p-value']

We see that the 'p-value' for store_42 is less that 0.05 while for store_30 it is 0.025. Looking at the result above p-Value is < 0.05 and Test Statistic is < Critical Value for store_42

### Implementation 

* We will use Box-Jenkins methodology to work on and assess our model.
* As we are forcasting the sales of different stores (by summing up the sales of all of their departments) and we have time series, and after examing our data we decided to use AR model. In order to consider external variables we will need to use Linear Regression.Therefore, we worte these custom functions below (fit_model_ar_with_external_variables(), predict_model_ar_with_external_variables()) to consider both Autoregression and Linear regression

* As the datasets we used are from 2010 - 2012, we will train our model on 2010-2011 and do the prediction on 2012

### Autoregression Time Series Model

#### Functions for Model Fit and Test

In [21]:
def fit_model_ar_with_external_variables(time_series, order_array, external_variables):

    data_x = []
    for index in range(np.max(order_array), len(time_series)):
            
            data_x.append(time_series.values[(index-order_array)].squeeze())
    
    data_x = np.array(data_x)
    data_x = np.append(data_x, external_variables.values[np.max(order_array):], axis=1)
    data_y = time_series[-len(data_x):].values
    lr_model=LinearRegression()
    lr_model.fit(data_x,data_y.ravel())
    print('Model Score: %.2f' % lr_model.score(data_x,data_y))
    print("coefficients:", lr_model.coef_)
    return lr_model.coef_, lr_model.intercept_

def predict_model_ar_with_external_variables(time_series, order_array, external_variables, coef, intercept):
    
    data_x = []
    for index in range(np.max(order_array), len(time_series)):
            
        data_x.append(time_series.values[(index-order_array)].squeeze())
    
    data_x = np.array(data_x)
    data_x = np.append(data_x, external_variables.values[np.max(order_array):], axis=1)
    data_x = np.array(np.dot(data_x, coef.T) + intercept)
    return data_x

def fit_model_ar(time_series, order_array):

    data_x = []
    for index in range(np.max(order_array), len(time_series)):
    
        data_x.append(time_series.values[(index-order_array)].squeeze())

    data_x = np.array(data_x)
    data_y = time_series[-len(data_x):].values
    lr_model=LinearRegression()
    lr_model.fit(data_x,data_y)
    print('Model Score: %.2f' % lr_model.score(data_x,data_y))
    print("coefficients:", lr_model.coef_)
    return lr_model.coef_, lr_model.intercept_

def predict_model_ar(time_series, order_array, coef, intercept):
    
    data = []
    for index in range(np.max(order_array), len(time_series)):
        
        data.append(np.sum(np.dot(coef, time_series.values[(index-order_array)].squeeze())) + intercept)

    return np.array(data)

In [22]:
###Plot for actual and predicted values.

def plot_predicted_and_actual_sales(actual, prediction):

    plt.figure(figsize=(20,5))
    plt.plot(actual, label='Actual Sales')
    plt.plot(prediction, label='Predicted Sales')
    _ = plt.ylabel("Sales ($)", fontsize = 15)
    _ = plt.xlabel("Date", fontsize = 15)
    _ = plt.legend(fontsize = 15)
    plt.legend()
    plt.show()

#### Functions for Evaluation 

In [23]:
def calculate_and_plot_AR_Residuals(actual, prediction):
    
    difference=(actual['Weekly_Sales']-prediction[0])/actual['Weekly_Sales']
    print('Residuals: Mean %.2f, std %.2f' % (difference.mean(), difference.std()))
    plt.figure(figsize=(20,5))
    plt.plot(difference, c='green')
    _ = plt.ylabel("Sales ($)", fontsize = 15)
    _ = plt.xlabel("Date", fontsize = 15)
    plt.grid()
    plt.show()

In [24]:
def calculate_diff_of_last_five_weeks(actual, prediticed):
    
    for i in range(5):
        print("Actual           ", "##############", "Predicted           ")
        print(float(actual["Weekly_Sales"][(len(actual) - 1) - i]),"##############", float(prediticed[0][(len(prediticed) - 1) - i]))
    print()
    for i in range(5):
        print("Difference        ", "##############", "Precentage           ")
        act = float(actual["Weekly_Sales"][(len(actual) - 1) - i])
        pred_num = float(prediticed[0][(len(prediticed) - 1) - i])
        diff = act - pred_num
        percent = (diff * 100)/act
        print(diff, " ##############", percent)

#### Modelling and Analysis for Stationary Data (Store_42)

During the EDA we saw that the store_42 passed the stationary test. Hence we start off by modelling our first time series model using this store

* We will start off by plotting ACF and PACF for the store by using the functions written earlier. 

In [ ]:
store_42_ts = get_Weekly_sales_as_time_series_by_store_id(42, df)
store_42_external_variables = get_external_variables_by_store_id(42, df)
plot_sales(store_42_ts)
plot_acf_and_pacf(store_42_ts)

* worth mentioning: our data has 143 weeks out of which 43 are related to 2012

In [ ]:
print(len(store_42_ts))
display(store_42_ts.iloc[:100])
display(store_42_ts.iloc[100:])

* looking at the tables above we will notice that 2012 starts from week number 100

* we will do fitting and predictiong (without external variables) based on the result above. In PACF result we noticed that our significant values are
1,3,46,60,69

In [ ]:
# we got these numbers from our observation above, but we will add 100 at the end as our prediciton function will start from the
# last week (biggest number) which will be 100 in this case
significants=np.array([1,3,46,60,69, 100])
coef, intercept = fit_model_ar(store_42_ts,significants)
pred=pd.DataFrame(index=store_42_ts[100:].index, data=predict_model_ar(store_42_ts, significants, coef, intercept))

- our score factor is 0.27. The perfect score factore is 1.0. Now we will plot a comparision between actual and predicted data

In [ ]:
# # 2012 starts from week 100
plot_predicted_and_actual_sales(store_42_ts[100:], pred)

* we will now apply the same steps (using external variables) and see if there is a significant difference

In [ ]:
external_variables_df = get_external_variables_by_store_id(42, df)
coef, intercept = fit_model_ar_with_external_variables(store_42_ts,significants, external_variables_df)
pred_with_ext=pd.DataFrame(index=store_42_ts[100:].index, data=predict_model_ar_with_external_variables(store_42_ts, significants,external_variables_df, coef, intercept))

- looking at the difference in score factore (0.27 without ext variables) and (0.56 with ext variables). The difference is big; therefore, considering external variables is important

#### Comparision between actual and predicted data using both with and without ext variables for store_42

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(store_42_ts[100:], label='Actual Sales')
plt.plot(pred, label='Predicted Sales (without ext variabales)')
plt.plot(pred_with_ext, label='Predicted Sales (with ext variabales)')
plt.legend()
plt.show()


* As shown above considering external variables is way more accurate. Therefore, we will consider fitting and modeling the next  stores without testing the prediction using no external variables as what we have done.

#### Evaluation of the AR model for store_42

* We will now calculate the AR residual of store number 42

In [ ]:
calculate_and_plot_AR_Residuals(store_42_ts[100:], pred_with_ext)

* printing the difference of the last five weeks

In [ ]:
calculate_diff_of_last_five_weeks(store_42_ts[100:], pred_with_ext)

#### RMSE Error for Store_42

In [ ]:
rms_store_42 = np.sqrt(mean_squared_error(store_42_ts[100:],pred_with_ext))
print('RMSE Error on Test data for Store 42:',rms_store_42)

* Now we will repeat the last 2 phases (Modeling and results) and (Evaluation) for store number 17, just to test our model on another stationary store


#### Modelling and Analysis for Stationary Data (Store_17)

In [ ]:
store_17_ts = get_Weekly_sales_as_time_series_by_store_id(17, df)
store_17_external_variables = get_external_variables_by_store_id(17, df)
plot_sales(store_17_ts)

* applying Dicky-Fuller test as a further check

In [ ]:
adf_test(store_17_ts["Weekly_Sales"])

* According to Dicky-Fuller test the data is stationary, except the fact that we have 2 trends that we're going to solve by consedring significants in PACF chart and then using our 2 custome fitting and predicting

* Plotting ACF and PACF

In [ ]:
plot_acf_and_pacf(store_17_ts)

* we will do fitting and prediction based on the result above. In PACF result we noticed that our significant values are
2,4,56,59,69,70

In [ ]:
significants=np.array([2,4,45,56,69,70,100])
external_variables_df = get_external_variables_by_store_id(17, df)
coef, intercept = fit_model_ar_with_external_variables(store_17_ts,significants, external_variables_df)
pred_with_ext=pd.DataFrame(index=store_17_ts[100:].index, data=predict_model_ar_with_external_variables(store_17_ts, significants,external_variables_df, coef, intercept))

We have included the value 100 in our significants too since our prediciton function will start from this week and we have observed on trial and error basis that when including the last week the model gives better results. 

- our score factor is 0.6. The perfect score factore is 1.0. Now we will plot a comparision between actual and predicted data

In [ ]:
# 2012 starts from week 100
plot_predicted_and_actual_sales(store_17_ts[100:], pred_with_ext)

#### Comparision between actual and predicted data using both with and without ext variables for store_17

* We will now calculate the AR residual of store number 17

In [ ]:
calculate_and_plot_AR_Residuals(store_17_ts[100:], pred_with_ext)

* printing the difference of the last five weeks

#### RMSE Error for Store_17

In [ ]:
rms_store_17 = np.sqrt(mean_squared_error(store_17_ts[100:],pred_with_ext))
print('RMSE Error on test data for store 17:',rms_store_17)

In [ ]:
calculate_diff_of_last_five_weeks(store_17_ts[100:], pred_with_ext)

### Working with Non-Stationary Data

#### Modelling and Analysis for Non-Stationary Data (Store_30)

From the EDA done earlier we have seen that store_30 was non-stationary as per the adf test. 

In [ ]:
store_30_ts = get_Weekly_sales_as_time_series_by_store_id(30, df)
external_variables_df = get_external_variables_by_store_id(30, df)
plot_sales(store_30_ts)

In [ ]:
adf_test(store_30_ts["Weekly_Sales"])

We see that after 3 lags the data is close to stationary as per the ADF test. We will try to take single dif transformation and check for the stationarity property

In [ ]:
store_30_diff = store_30_ts.diff().fillna(0)
plot_sales(store_30_diff)

In [ ]:
adf_test(store_30_diff["Weekly_Sales"])

The data is now close to stationary from the plot we see that rolling mean is almost a flat curve. We will plot pacf and acf for this data

In [ ]:
plot_acf_and_pacf(store_30_ts)

In [ ]:
significants=np.array([1,2,3,4,59,60,61,62,100])
coef, intercept = fit_model_ar_with_external_variables(store_30_diff,significants,external_variables_df)
pred_with_ext=pd.DataFrame(index=store_30_diff[100:].index, data=predict_model_ar_with_external_variables(store_30_diff, significants,external_variables_df, coef, intercept))

#### Comparision between actual and predicted data using both with and without ext variables for store_30

In [ ]:
plot_predicted_and_actual_sales(store_30_diff[100:], pred_with_ext)

In [ ]:
calculate_and_plot_AR_Residuals(store_30_diff[100:], pred_with_ext)

We see that the model has got a good score and fit well after we have transformed the data to a single diffed series. 

In [ ]:
calculate_diff_of_last_five_weeks(store_30_diff[100:], pred_with_ext)

#### RMSE for Store_30_diff

In [51]:
rms_store_30 = np.sqrt(mean_squared_error(store_30_diff[100:],pred_with_ext))


In [ ]:
print('RMSE Error on Test data for diffed Store 30:',rms_store_30)

### Approach using Classical Time-Series Model ARIMA

The idea with this part of the implementation is to forecast on sales values without using the external variables(keeping it uni-variate) and considering integrated moving average value and AR values. 

We first approach this problem using AUTO ARIMA built-in function for grid search to find the best p,d,q parameters and then use them on the statsmodels ARIMA function and predict sales values on the test set. 

In [53]:
#### First we Define the Functions to model our data and visualize the same. 

In [54]:
from statsmodels.tsa.arima_model import ARIMA
def ARIMA_model(time_series,arima_order):
    '''This functions takes the time series input, makes predictions on walk through validation basis and returns the predic
    tions and updated training set,'''
    X = time_series.values
    train, test = X[0:100], X[100:]
    train_updated = [x for x in train]
    predictions = list()
    
    for i in range(len(test)):
        model = ARIMA(train_updated, order=arima_order)
        ARIMA_results = model.fit(trend='nc', disp=False)
        yhat = ARIMA_results.forecast()[0]
        predictions.append(yhat)
        act = test[i]
        train_updated.append(act)
    return (predictions,train_updated)


def ARIMA_Visualize(time_series, predicted, trained):
    '''This function takes the input of original time series, the predicted and the updated trained values
    and returns plots to compare the actual test series with the  predicted values'''
    predicted_series= pd.Series(predicted, index=time_series.index[100:])
    trained_series= pd.Series(trained, index=time_series.index[:len(time_series)])
    plt.figure(figsize=(12,5))
    plt.plot(time_series[100:], label='Test')
    #plt.plot(trained_series, label='Walk Throgh Validated')
    plt.plot(predicted_series, label='predicted')
    plt.legend(loc='upper left', fontsize=8)
    plt.title('Forecast vs Actuals')
    plt.show()
    return None

#### ARIMA for store_42

As with Auto-regression we use the same stores to check for ARIMA and then evaluate using RMSE values.   

#### Grid search to find the optimal parameters

In [ ]:
import pmdarima as pm
store_42_ts = get_Weekly_sales_as_time_series_by_store_id(42,df)
stepwise_fit = pm.auto_arima(store_42_ts, start_p=1, start_q=1, max_p=5,max_q=5,m=6,
                             start_P=0, seasonal=False, d=1, D=1, trace=True,
                             error_action='ignore',  # don't want to know if an order does not work
                             suppress_warnings=True,  # don't want convergence warnings
                             stepwise=True)  # set to stepwise

stepwise_fit.summary()

In [ ]:
#### Run the model and apply the visualization
predicted_values = ARIMA_model(store_42_ts,(0,1,2))[0]
ARIMA_Visualize(store_42_ts, predicted_values, ARIMA_model(store_42_ts,(0,1,2))[1])

#### RMSE for store_42 with ARIMA

In [ ]:
rmse_store_42_ARIMA = np.sqrt(mean_squared_error(store_42_ts[100:],predicted_values))
print('RMSE Error on Test data for Store 42:',rmse_store_42_ARIMA)

#### ARIMA for store_17

#### Grid search to find the optimal parameters

In [ ]:
import pmdarima as pm
store_17_ts = get_Weekly_sales_as_time_series_by_store_id(17,df)
stepwise_fit = pm.auto_arima(store_17_ts, start_p=1, start_q=1, max_p=5,max_q=5,m=6,
                             start_P=0, seasonal=False, d=1, D=1, trace=True,
                             error_action='ignore',  # don't want to know if an order does not work
                             suppress_warnings=True,  # don't want convergence warnings
                             stepwise=True)  # set to stepwise

stepwise_fit.summary()

In [ ]:
#### Run the model and apply the visualization
predicted_values = ARIMA_model(store_17_ts,(1,1,2))[0]
ARIMA_Visualize(store_17_ts, predicted_values, ARIMA_model(store_17_ts,(1,1,2))[1])

#### RMSE for store_17



In [ ]:
rmse_store_17 = np.sqrt(mean_squared_error(store_17_ts[100:],predicted_values))
print('RMSE Error on Test data for diffed Store 17:',rmse_store_17)

#### ARIMA for Store_30

In [ ]:
import pmdarima as pm
store_30_ts = get_Weekly_sales_as_time_series_by_store_id(30,df)
stepwise_fit = pm.auto_arima(store_30_ts, start_p=1, start_q=1, max_p=5,max_q=5,m=6,
                             start_P=0, seasonal=False, d=1, D=1, trace=True,
                             error_action='ignore',  # don't want to know if an order does not work
                             suppress_warnings=True,  # don't want convergence warnings
                             stepwise=True)  # set to stepwise

stepwise_fit.summary()

In [ ]:
#### Run the model and apply the visualization
predicted_values = ARIMA_model(store_30_ts,(1,1,0))[0]
ARIMA_Visualize(store_30_ts, predicted_values, ARIMA_model(store_30_ts,(1,1,0))[1])

In [ ]:
rmse_store_30 = np.sqrt(mean_squared_error(store_30_ts[100:],predicted_values))
print('RMSE Error on Test data for Store 30:',rmse_store_30)